# 5. Reducción dataset

El objetivo de este notebook es procesar el dataset que se ha utilizado para entrenar le modelo preliminar y tratar de reducir la dimensionalidad de este sin perder precisión. De esta forma, se identificarán que sensores no son necesarios en la máquina para identificar posibles reducciones de costes.

## 5.1. Dataset y modelo inicial

In [59]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

def evaluar_precision(modelo, X, y, cv=5):
    """
    Entrena un modelo usando cross-validation y retorna la precisión media.
    :param modelo: instancia de un estimador de scikit-learn (por ej. RandomForestClassifier).
    :param X: features (DataFrame o NumPy array).
    :param y: variable objetivo (Series o array).
    :param cv: número de folds para la validación cruzada.
    :return: None (imprime la precisión media y desviación estándar).
    """
    scores = cross_val_score(modelo, X, y, cv=cv, scoring='accuracy')
    print(f"Precisión media (CV={cv}): {scores.mean():.4f} ± {scores.std():.4f}")


def evaluar_metricas(modelo, X, y, cv=5):
    """
    Entrena un modelo usando cross-validation y reporta Accuracy, Recall y F1.
    """
    scoring = {
        'accuracy': 'accuracy',
        'recall': 'recall_macro',   # o 'recall_weighted' si lo prefieres
        'f1': 'f1_macro'            # o 'f1_weighted'
    }
    resultados = cross_validate(modelo, X, y, cv=cv, scoring=scoring)
    print(f"Accuracy (CV={cv}): {resultados['test_accuracy'].mean():.4f} ± {resultados['test_accuracy'].std():.4f}")
    print(f"Recall   (CV={cv}): {resultados['test_recall'].mean():.4f} ± {resultados['test_recall'].std():.4f}")
    print(f"F1       (CV={cv}): {resultados['test_f1'].mean():.4f} ± {resultados['test_f1'].std():.4f}")


In [60]:
df = pd.read_csv("processed_data_ultimate.csv")
print("El dataset tiene incialmente un tamaño de:")
df.shape

El dataset tiene incialmente un tamaño de:


(84000, 67)

Antes de comenzar la reducción de dimensionalidad del dataset, el modelo de Random Forest tiene una precisión de:


In [61]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

X = df.drop(columns=['Tipo', 'Hz', 'medida'])  # Ejemplo, quitamos las que no van al modelo
y = df['Tipo']

modelo = RandomForestClassifier(n_estimators=100, random_state=42)

print("=== Métricas sin reducción de dimensionalidad ===")
evaluar_precision(modelo, X, y)  # o evaluar_metricas(modelo, X, y)


=== Métricas sin reducción de dimensionalidad ===
Precisión media (CV=5): 0.9048 ± 0.0473


## 5.2. Filtrado inicial
Reducción de dimensionalidad no supervisada
### 5.2.1. Eliminar variables de baja varianza
Algunas variables pueden presentar muy poca variación (casi todos los valores iguales), lo que las hace poco relevantes para la predicción.

In [62]:
from sklearn.feature_selection import VarianceThreshold

# Umbral de varianza muy pequeño, por ejemplo 0.01
selector_var = VarianceThreshold(threshold=0.01)
X_var_reduced = selector_var.fit_transform(X)

print("\n=== Métricas tras eliminar varianza muy baja ===")
evaluar_precision(modelo, X_var_reduced, y)


=== Métricas tras eliminar varianza muy baja ===
Precisión media (CV=5): 0.9011 ± 0.0516


### 5.2.2. Análisis de correlación
Si hay variables altamente correlacionadas entre sí (multicolinealidad), puede bastar con quedarte con una de ellas.

In [63]:
import numpy as np

# OJO: para esto necesitamos trabajar sobre DataFrame; si es un numpy array, 
# conviene primero seleccionar las columnas "supervivientes" de VarianceThreshold.
columns_survived = X.columns[selector_var.get_support()]
X_temp = X[columns_survived]

# Matriz de correlaciones
corr_matrix = X_temp.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Umbral de 0.9, por ejemplo
to_drop = [c for c in upper_tri.columns if any(upper_tri[c] > 0.9)]
X_corr_reduced = X_temp.drop(columns=to_drop)

print("\n=== Métricas tras eliminar correlaciones altas ===")
evaluar_precision(modelo, X_corr_reduced, y)



=== Métricas tras eliminar correlaciones altas ===
Precisión media (CV=5): 0.8992 ± 0.0431


## 5.3. Métodos supervisados de selección de características
Estos métodos consideran la relación entre cada variable y la variable objetivo ("Tipo") para descartar características irrelevantes o redundantes.

### 5.3.1. Feature importance en modelos basados en árboles
* Entrenar un Random Forest o un XGBoost preliminarmente y obtener la importancia de cada variable.
* Con la importancia, filtrar aquellaspor debajo un cierto umbral o seleccionar las "top K" variables más relevantes.

In [64]:
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import LabelEncoder
# from xgboost import XGBClassifier

# le = LabelEncoder()
# y_train_encoded = le.fit_transform(y_train)
# y_test_encoded = le.transform(y_test)   # para test, usar "transform", NO "fit_transform"

# model = XGBClassifier()
# model.fit(X_train, y_train_encoded)

# y_pred_encoded = model.predict(X_test)

# y_pred = le.inverse_transform(y_pred_encoded) # Descodificar para obtener los nombres originales

# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# print("\nAccuracy:", accuracy_score(y_test, y_pred))
# print("Matriz de Confusión:\n", confusion_matrix(y_test, y_pred))
# print("Reporte de Clasificación:\n", classification_report(y_test, y_pred))


### 5.3.2. Selección por modelos lineales con regularización L1 (Lasso)
* Aplicar un modelo lineal (por ejemplo, Logistic Regression con penalización L1) que fuerce a ciertos coeficientes a ser cero y elimina variables poco relevantes.

In [65]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.feature_selection import SelectFromModel

# estimator = LogisticRegression(penalty='l1', solver='saga', max_iter=84000)
# selector = SelectFromModel(estimator=estimator, threshold='mean')
# selector.fit(X_train, y_train)

# X_train_reduced = selector.transform(X_train)
# X_test_reduced  = selector.transform(X_test)

In [66]:
# print("Tamaño después de procesar")
# print(X_train_reduced.shape)
# print(X_test_reduced.shape)

### 5.3.3. Wrapper methods: Recursive Feature Elimination (RFE)
* Consiste en entrenar un modelo y eliminar iterativamente las características 

In [67]:
# from sklearn.feature_selection import RFE
# from sklearn.ensemble import RandomForestClassifier

# estimator = RandomForestClassifier(n_estimators=100)
# rfe = RFE (estimator, n_features_to_select=20)
# rfe.fit(X_train, y_train)

# X_train_reduced = rfe.transform(X_train)
# X_test_reduced = rfe.transform(X_test)

## 5.4. Métodos de proyección
En estos casos, las variables originales se combinan (mediante proyecciones lineales o no lineales) para reducir la dimensión. Algunas técnicas son:

### 5.4.1. Principal component Analysis (PCA)
* Método no supervisado que busca maximizar la varianza en los primeros componentes principales.
* Útil cuando hay mucho colinealidad y el dataset es muy grande.
* El inconveniente es que las nuevas variables (componentes principales) pueden perder interpretabilidad directa.

In [ ]:
from sklearn.decomposition import PCA

# Por ejemplo, 10 componentes
pca = PCA(n_components=10, random_state=42)
X_pca = pca.fit_transform(X_corr_reduced)

print("\n=== Métricas tras PCA a 10 componentes ===")
evaluar_precision(modelo, X_pca, y)



=== Métricas tras PCA a 10 componentes ===


### 5.4.2. Linear Discriminant Analysis (LDA)
* Similar al PCA per supervisado.
* Para problemas multiclass, LDA puede reducir la dimensionalidad hasta n_clases -1 componentes relevantes.
* Puede ser muy efectivo si las clases están bien separadas linealmente.

In [ ]:
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# lda = LDA(n_components = 2) # 2 componentes
# x_lda = lda.fit_transform(X,y)

## 5.5. Comprobar precisión después de reducción de dimensionalidad
Como conclusión, el flujo de técnicas que se han utilizado para la reducción de dimensionalidad es el siguiente:
1. Un filtrado inicial (varianza y correlación),
2. Método supervisado (por ejemplo, importancia de variables en Random Forest / XGBoost, o un modelo lineal con L1)
3. PCA o LDA

In [ ]:

evaluar_precision(modelo, X_pca, y)
evaluar_metricas(modelo, X_pca, y)